#### Import necessary libraries for data handling and visualization.

In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import matplotlib.ticker as mticker

#### Load csv files with data from various CEO Validation (EPR 4000 points, 1st Validation 835 points, 2nd Validation 3467 points)

In [11]:
pts835 = pd.read_csv('/home/sepal-user/eSBAE_CIV/835_final.csv', delimiter=';')
pts3k = pd.read_csv('/home/sepal-user/eSBAE_CIV/3K_final.csv', delimiter=';')
erp4k = pd.read_csv('/home/sepal-user/eSBAE_CIV/erp4K_final.csv', delimiter=';')
df320k = pd.read_csv('/home/sepal-user/eSBAE_CIV/320Kupdate.csv', delimiter=',')

#### Merge pts835 and pts3k into a new dataframe > {merged_national} = 4302 points / which could be use as training dataset

In [5]:
merged_national = pd.concat([pts835, pts3k])
len(merged_national)

4302

#### Load the country 1km grid csv or gpkg file with all the points and columns including strata or kmeans, chg_prob etc.. 
#### For Ivory Coast we have 325631 points 

In [19]:
#df320k = gpd.read_file('/home/sepal-user/module_results/esbae/Cote_Ivoire_MRV/cote_ivoire_all_classified_20231114.gpkg')
len(df320k)

325631

In [ ]:
#df320k.to_parquet('/home/sepal-user/module_results/esbae/Cote_Ivoire_MRV/cote_ivoire_all_classified_20231114.parquet')

In [12]:
# List all columns 
df320k.columns.tolist()

['fid',
 'point_id',
 'images',
 'mon_images',
 'bfast_change_date',
 'bfast_magnitude',
 'bfast_means',
 'cusum_change_date',
 'cusum_confidence',
 'cusum_magnitude',
 'red_mean',
 'red_sd',
 'red_min',
 'red_max',
 'nir_mean',
 'nir_sd',
 'nir_min',
 'nir_max',
 'swir1_mean',
 'swir1_sd',
 'swir1_min',
 'swir1_max',
 'swir2_mean',
 'swir2_sd',
 'swir2_min',
 'swir2_max',
 'ndfi_mean',
 'ndfi_sd',
 'ndfi_min',
 'ndfi_max',
 'brightness_mean',
 'brightness_sd',
 'brightness_min',
 'brightness_max',
 'greenness_mean',
 'greenness_sd',
 'greenness_min',
 'greenness_max',
 'wetness_mean',
 'wetness_sd',
 'wetness_min',
 'wetness_max',
 'bs_slope_mean',
 'bs_slope_sd',
 'bs_slope_max',
 'bs_slope_min',
 'ewma_jrc_date',
 'ewma_jrc_change',
 'ewma_jrc_magnitude',
 'mosum_jrc_date',
 'mosum_jrc_change',
 'mosum_jrc_magnitude',
 'cusum_jrc_date',
 'cusum_jrc_change',
 'cusum_jrc_magnitude',
 'ccdc_change_date',
 'ccdc_magnitude',
 'aspect',
 'dw_class_mode',
 'dw_tree_prob__max',
 'dw_tree_pr

In [13]:
column_of_interest = 'ocs_2020' # your classes/redd activities/target column that contains the classes for which you want to get the area estimations

In [14]:
# merge your full database columns, point_id, kmeans with your 3000 thousand points

In [15]:
df_merged = df320k[['point_id', 'kmeans_multi']].merge(pts3k[['point_id', column_of_interest]], how='left', on='point_id')

In [16]:
np.unique(df_merged.kmeans_multi, return_counts=True)

(array([0, 1, 2]), array([ 95390,  34188, 196053]))

#### Perform area calculation using the stratum column. In this case the column is called kmeans. Use the merge dataframe (national grid points + CEO validated points)

In [17]:
def calculate_areas(df_merged, kozak_column, target_column, total_area, z_score):
    
    df_full = df_merged.copy()
    df_merged = df_merged[~df_merged[target_column].isna()]
    
    # get all attributes
    categories = df_merged[target_column].unique()
    
    # get strata
    strata, d = df_merged[kozak_column].unique(), {}
    print(categories)
    # create stats for each entry
    for category in categories:
        
        if str(category) == 'nan':
            continue
            
        print(f' Calculating stats for {category}')
        # create binary class column
        df_merged[category] = df_merged[target_column].apply(lambda x: 1 if x == category else 0)
        print(f'There are {df_merged[category].sum()} entries of {category} in {target_column}.')
        
        # initialize variables for category <> check the catergories > def, deg, gain
        categories_area, se_total = 0, 0
        d2 = {}
        for kmeans in strata:
                        
            if str(kmeans) == 'nan':
                continue
            
            # subset to stratum
            kmeans_df = df_full[df_full[kozak_column] == kmeans]
            
            # get area proportion for that stratum on full dataset
            proportion_strata = len(kmeans_df)/len(df_full)

            # get stratum area
            stratum_area = proportion_strata * total_area

            # get proportion of forest change within strata from interpreted data
            proportion_category = len(
                df_merged[(df_merged[kozak_column] == kmeans) & (df_merged[category] == 1)]
            ) / len(
                df_merged[df_merged[kozak_column] == kmeans]
            )
            
            # get area from proportion and full area
            area = proportion_category * stratum_area

            # get error from interpreted data for full stratum area
            #var = np.var(df_merged[category][df_merged[kozak_column] == kmeans]) / len(df_merged[df_merged[kozak_column] == kmeans])
            #se = np.sqrt(var) * stratum_area

            var = np.var(df_merged[category][df_merged[kozak_column] == kmeans])
            sd = np.sqrt(var)               
            n = len(df_merged[df_merged[kozak_column] == kmeans])
            se = sd/np.sqrt(n) * stratum_area
                          
            # add for totals
            categories_area += area
            se_total += se**2
            
            # add to dictionary
            d2[f'area_stratum_{kmeans}'] = area
            d2[f'ci_stratum_{kmeans}']=1.67*se
        
        d2['area_total'] = categories_area
        d2['ci_total'] = z_score*np.sqrt(se_total)
        d2['relative CI'] =  z_score*np.sqrt(se_total) / categories_area * 100
        d[category] = d2
    
    
    return pd.DataFrame.from_dict(d, orient='index')

In [18]:
calculate_areas(df_merged, 'kmeans_multi', column_of_interest, total_area=len(df_merged), z_score=1.645)

['Terres gramineennes' 'Terres cultivees' 'Terres forestieres'
 'Etablissement humain' 'Terres humides' 'Autres terres']
 Calculating stats for Terres gramineennes
There are 545 entries of Terres gramineennes in ocs_2020.
 Calculating stats for Terres cultivees
There are 1926 entries of Terres cultivees in ocs_2020.
 Calculating stats for Terres forestieres
There are 919 entries of Terres forestieres in ocs_2020.
 Calculating stats for Etablissement humain
There are 25 entries of Etablissement humain in ocs_2020.
 Calculating stats for Terres humides
There are 40 entries of Terres humides in ocs_2020.
 Calculating stats for Autres terres
There are 12 entries of Autres terres in ocs_2020.


/tmp/ipykernel_972/449317470.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged[category] = df_merged[target_column].apply(lambda x: 1 if x == category else 0)
/tmp/ipykernel_972/449317470.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged[category] = df_merged[target_column].apply(lambda x: 1 if x == category else 0)
/tmp/ipykernel_972/449317470.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

,area_stratum_0,ci_stratum_0,area_stratum_2,ci_stratum_2,area_stratum_1,ci_stratum_1,area_total,ci_total,relative CI
Terres gramineennes,18022.258245,1474.333947,36870.327703,4293.350874,1731.037975,445.358922,56623.623922,4492.953272,7.934768
Terres cultivees,50440.994969,1880.023009,119662.979730,5358.067832,18954.865823,1009.619933,189058.840522,5681.045200,3.004909
Terres forestieres,25487.098938,1666.552898,30688.476351,3992.260058,13069.336709,987.104000,69244.911998,4370.904648,6.312239
Etablissement humain,213.281163,177.890565,4194.827703,1589.853852,86.551899,102.076938,4494.660764,1579.030961,35.131260
Terres humides,1119.726104,405.652902,2428.584459,1215.252866,346.207595,203.375156,3894.518158,1277.791249,32.809996
Autres terres,106.640581,125.858074,2207.804054,1159.358559,0.000000,0.000000,2314.444635,1148.712389,49.632312
